<a href="https://colab.research.google.com/github/paulomrt/Python-DataProjects/blob/main/Vendas%20de%20Fertilizantes%20de%20uma%20Empresa%20(C%C3%B3digo%20Exemplo%20de%20Limpeza%20e%20Visualiza%C3%A7%C3%A3o).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise da quantidade de fertilizantes vendidos no ano de 2021 de uma empresa

Para fins de mostrar meu conhecimento utilizando python para o tratamento de dados, esse é um código que escrevi e utilizei 
para a recuperação e limpeza de dados de uma empresa em que trabalhei. 

Os pontos abordados nessa análise serão:

*   Mostrar um gráfico do histórico de vendas dos fertilizantes mês a mês.
*   Quais foram os fertilizantes mais vendidos durante o ano todo?
*   Quais foram os fertilizantes mais vendidos em cada trimestre e sua média trimestral?
*   Quais foram os 3 meses que mais venderam fertilizantes?

Observação: **`Deixo claro que nenhum dado privado será mostrado, somente o código utilizado. `**

In [ ]:
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go

## Importandos os dados
Importação de todas as planilhas de cada mês do ano, sendo concatenadas em um único arquivo .csv chamado "all_months".

Além disso, é feita a criação de uma coluna chamada "Month", no qual seus valores são os respectivos nomes de cada arquivo.

In [ ]:
#@title
# Criação de uma lista com o nome de cada arquivo dentro da pasta "All_Months"
path = "/content/drive/MyDrive/Colab_Notebooks/Data_Folder"
files = [file for file in os.listdir(path) if not file.startswith('.')]

# Utilização de um loop for para criação de um dataframe e a concatenação de
# de todos os arquivos em um só único.
df = pd.DataFrame()

for file in files:
    current_data = pd.read_excel(path+"/"+file)
    current_data['Mês'] = file
    df = pd.concat([df, current_data])

# Exportando esse novo dataframe como um arquivo .csv    
df.to_csv(
    "/content/drive/MyDrive/Colab_Notebooks/Data_Folder/sales.csv", 
    index=False)

# Montagem do dataframe através desse arquivo .csv
df = pd.read_csv(
    "/content/drive/MyDrive/Colab_Notebooks//sales.csv")

## Limpeza e tratamento geral dos dados
Nessa parte é feita a limpeza geral dos dados, seleção das colunas interessadas, remoção de NaN e outras linhas que não são dados de interesse da análise.

### Splitando o valor .xls da coluna "Mês"
Aqui estarei removendo somente a parte ".xls" dos valores da coluna "Mês".

In [ ]:
df["Mês"] = df["Mês"].str.replace(".xls", "")

### Convertendo a coluna "Mês" do tipo string para datetime
Simples converção de tipo de valor da coluna "Month", indo do tipo string para o tipo datetime.

In [ ]:
df["Mês"] =  pd.to_datetime(df["Mês"], format='%B-%y')

### Seleção das colunas interessadas
Aqui é feita a seleção apenas das colunas que nos interessa para a análise, sendo elas: **"Unnamed: 0"**,	**"Unnamed: 9"**,	**"Unnamed: 10"** e	**"Mês"**


In [ ]:
df = df[["Unnamed: 0", "Unnamed: 9", "Unnamed: 10", "Mês"]]

### Remoção de linhas que não começam com "PA."
Todos os valores da coluna "Unnamed: 0" que não começam com "PA." são removidos do dataframe.

In [ ]:
df = df[all_months["Unnamed: 0"].str.startswith("PA.", na=False)]

### Remoção de espaços e aplicação de "upper" na coluna "Unnamed: 0"
Aqui é feito a remoção dos espaços entre os nomes dos produtos e o "aumento" das letras para que todas fiquem em caixa alta.


In [ ]:
df["Unnamed: 0"] = df["Unnamed: 0"].str.upper().str.replace(' ', '')

### Renomear colunas
Aqui é feito a renomeação das colunas "Unnamed: 0,	Unnamed: 9 e	Unnamed: 10"

In [ ]:
df = df.rename(columns={'Unnamed: 0': 'Produto',
                                        'Unnamed: 9': 'Quantidade (L)',
                                        'Unnamed: 10': 'Valor Total (R$)'})

### Conversão do tipo das colunas "Quantidade (L)" e "Valor Total (R$)"
Realizada a conversão das colunas para o tipo numéricas (int e float).

In [ ]:
df[["Quantidade (L)", "Valor Total (R$)"]] = df[["Quantidade (L)", "Valor Total (R$)"]].apply(pd.to_numeric)
df = all_months.reset_index(drop=True)

## Separação dos DataFrames para Análise



### Preparar o dataframe para análise da quantidade de fertilizantes mês a mês
Remoção dos caracteres que separam cada um dos fertilizantes por embalagem utilizando Expressões Regulares.


In [ ]:
df["Produto"] = df["Produto"].str.extract(r"^.*?\.([^.]*)\.")

### Agrupamento dos fertilizantes mês a mês e ordenar por suas quantidades totais
Código que faz o agrupamento dos fertilizantes mês a mê em conjunto com uma função que faz seu ordenamento baseado na quantidade total, sendo do maior pro menor.

In [ ]:
# Agrupando por cada fertilizantes com seus respectivos meses
gbdate_mtm = pd.DataFrame(df.groupby(["Produto", "Mês"])["Quantidade (L)"].sum()).reset_index()
gbdate_mtm

In [ ]:
# Agrupando por cada fertilizantes com seus respectivos meses
gbdate_mtm = pd.DataFrame(mtm_all_months.groupby(["Produto", "Mês"])["Quantidade (L)"].sum()).reset_index()

# Criação de um novo dataframe colocando em ordem decrescente pela quantidade somada do ano inteiro
df_products = pd.DataFrame(mtm_all_months.groupby(["Produto"])["Quantidade (L)"].sum()).sort_values("Quantidade (L)", ascending=False).reset_index()

# Criação de uma array em ordem decrescente dos fertilizantes que mais foram vendidos
products = df_products.head(10)['Produto'].unique()

# Uma nova coluna no df "gbdate_mtm" mas somente com aqueles fertilizantes que estão na array "products"
gbdate_mtm['Product_plot'] = gbdate_mtm['Produto'].apply(lambda x: x if x in products else None)

# Retirando todos "NaN" e resetando o index do dataframe "gbdate_mtm"
gbdate_mtm = gbdate_mtm[['Product_plot', 'Quantidade (L)', 'Mês']].dropna().reset_index(drop=True)

## Plotagem do gráfico do histórico de fertilizantes e suas quantidades mensal
Gráfico interativo em que mostra o histórico mensal de cada fertilizantes separados em subplots, junto com suas respectivas quantidades vendidas em cada mês. Gráfico feito utilizando o package "Plotly", o que possibilita uma interativade total e uma melhor visualização e análise dos resultados.

In [ ]:
# Criação do gráfico de barras separados por cada fertilizante
hist_mtm = px.bar(
              gbdate_mtm, 
              x='Mês', 
              y='Quantidade (L)', 
              color='Product_plot',
              #markers=True,
              text_auto = '.2s',
              # text='Quantidade (L)',
              # barmode = 'group',
              facet_col = 'Product_plot',
              facet_col_wrap= 5,
              facet_row_spacing = 0.25,
              facet_col_spacing = 0.02,
              #width = 2400,
              #height = 900,
              title='<b>Histórico mensal dos 10 fertilizantes mais vendidos em 2021</b>'
              )



# Configurações Gerais

# Alteração do tamanho e alinhamento do título do gráfico
hist_mtm.update_layout(title=dict(x=0.5, font=dict(size=24)))

# Remoção da string "Product_plot=" do título de cada subplot
hist_mtm.for_each_annotation(lambda a: a.update(text=a.text.replace("Product_plot=", "")))

# Define qual a localização do texto de cada barra
#hist_mtm.update_traces(textposition='outside')



# Configurações da legenda

# Esconder a legenda do gráfico
hist_mtm.update_traces(showlegend=False)
hist_mtm.update(layout_showlegend=False)



# Configurações do eixo x 

# Mudar o formato da data para somente o mês (%b)
hist_mtm.update_xaxes(tickformat="%b")

# Mostrar os meses em todos subplots
hist_mtm.update_xaxes(showticklabels=True)

# Mostrar sempre cada um dos meses
hist_mtm.update_xaxes(dtick="M1")

# Aumentar a distancia dos meses ao eixo x
hist_mtm.update_xaxes(linewidth = 3)

# Remoção do título do eixo x em cada subplot
hist_mtm.for_each_xaxis(lambda x: x.update(title = ''))

# Adiciona uma anotação que serve como o título do eixo x de todos subplots
hist_mtm.add_annotation(
            x=0.5,
            y=-0.2,
            font=dict(size=15),
            showarrow=False,
            text="<b>Meses</b>",
            textangle=-0,
            xref="paper",
            yref="paper")



# Configurações do eixo y

# Remoção do título do eixo y em cada subplot
hist_mtm.for_each_yaxis(lambda y: y.update(title = ''))

# Adiciona uma anotação que serve como o título do eixo y de todos subplots
hist_mtm.add_annotation(
            x= -0.06,
            y= 0.5,
            font=dict(size=15),
            showarrow=False,
            text="<b>Volume (L)</b>",
            textangle= -90,
            xref="paper",
            yref="paper")

hist_mtm.show()

## Quantidade de fertilizantes vendidos durante o ano de 2021

### Criação do dataframe ordenado pelos fertilizantes mais vendidos no ano
Dataframe criado somente com as colunas "Produto" e "Quantidade (L)" através do agrupamento de todos os produtos e a soma de suas quantidades.

In [ ]:
top_selling = pd.DataFrame(all_months.groupby(["Produto"])["Quantidade (L)"].sum()).sort_values("Quantidade (L)", ascending=False).reset_index()
top_selling.head(10)

### Gráfico da quantidade de fertilizantes vendidos em 2021

In [ ]:
# Criação do gráfico de barras separados por cada fertilizante
top_selling_graphic = px.bar(
              top_selling, 
              x='Produto', 
              y='Quantidade (L)', 
              text_auto = '.2s',
              title='<b>Quantidade de fertilizantes vendidos no ano de 2021</b>'
              )

# Alteração do tamanho e alinhamento do título do gráfico
top_selling_graphic.update_layout(title=dict(x=0.5, font=dict(size=24)))

top_selling_graphic.show()

## Fertilizantes vendidos no primeiro trimestre de 20021

### Quantidade vendida de cada fertilizante mês a mês do primeiro trimestre

In [ ]:
# Dataframe que mostra a quantidade de fertilizantes vendidos no primeiro trimestre
df_first_quarter = mtm_all_months
df_first_quarter = df_first_quarter[df_first_quarter["Mês"].dt.quarter == 1].reset_index(drop = True)

# Agrupar as quantidades vendidas em cada mês desse primeiro trimestre
df_first_quarter = pd.DataFrame(df_first_quarter.groupby(["Produto", "Mês"])["Quantidade (L)"].sum()).reset_index()

In [ ]:
# Criação do gráfico de barras separados por cada fertilizante e suas respectivas
# vendas em cada mês do primeiro trimestre
first_quarter = px.bar(
              df_first_quarter, 
              x='Mês', 
              y='Quantidade (L)', 
              color='Produto',
              text_auto = '.2s',
              facet_col = 'Produto',
              facet_col_wrap= 6,
              facet_row_spacing = 0.04,
              facet_col_spacing = 0.02,
              #width = 1200,
              height = 2400,
              title='<b>Vendas de fertilizantes mensal do primeiro trimestre de 2021</b>'
              )



# Configurações Gerais

# Alteração do tamanho e alinhamento do título do gráfico
first_quarter.update_layout(title=dict(x=0.5, font=dict(size=24)))

# Remoção da string "Product_plot=" do título de cada subplot
first_quarter.for_each_annotation(lambda a: a.update(text=a.text.replace("Produto=", "")))

# Define qual a localização do texto de cada barra
#first_quarter.update_traces(textposition='outside')



# Configurações da legenda

# Esconder a legenda do gráfico
first_quarter.update_traces(showlegend=False)
first_quarter.update(layout_showlegend=False)



# Configurações do eixo x 

# Mudar o formato da data para somente o mês (%b)
first_quarter.update_xaxes(tickformat="%b")

# Mostrar os meses em todos subplots
first_quarter.update_xaxes(showticklabels=True)

# Mostrar sempre cada um dos meses
first_quarter.update_xaxes(dtick="M1")

# Aumentar a distancia dos meses ao eixo x
first_quarter.update_xaxes(linewidth = 3)

# Remoção do título do eixo x em cada subplot
first_quarter.for_each_xaxis(lambda x: x.update(title = ''))



# Configurações do eixo y

# Remoção do título do eixo y em cada subplot
first_quarter.for_each_yaxis(lambda y: y.update(title = ''))

first_quarter.show()

### Média e soma de cada fertilizante vendido no primeiro trimestre
Criação de dois novos dataframes em que mostram as quantidades somadas de cada fertilizante vendido no primeiro trimestre e também a média de cada um deles.

In [ ]:
# Dataframe que mostra a quantidade vendida total de cada fertilizante no
# primeiro trimestre
df_first_quarter_sum = pd.DataFrame(df_first_quarter.groupby("Produto")["Quantidade (L)"].sum()).round()
df_first_quarter_sum = df_first_quarter_sum.sort_values("Quantidade (L)", ascending = False).reset_index()
df_first_quarter_sum.head(10)

In [ ]:
# Criação do gráfico de barras separados por cada fertilizante
first_quarter_sum_graphic = px.bar(
              df_first_quarter_sum, 
              x='Produto', 
              y='Quantidade (L)', 
              text_auto = '.2s',
              title='<b>Fertilizantes vendidos no primeiro trimestre de 2021</b>'
              )

# Alteração do tamanho e alinhamento do título do gráfico
first_quarter_sum_graphic.update_layout(title=dict(x=0.5, font=dict(size=24)))

first_quarter_sum_graphic.show()

In [ ]:
# Dataframe que mostra a média mensal vendida de cada fertilizante no
# primeiro trimestre
df_first_quarter_avg = pd.DataFrame(df_first_quarter.groupby("Produto")["Quantidade (L)"].mean()).round()
df_first_quarter_avg = df_first_quarter_avg.sort_values("Quantidade (L)", ascending = False).reset_index()
df_first_quarter_avg.head(10)

## Fertilizantes vendidos no segundo trimestre de 20021

In [ ]:
# Dataframe que mostra a quantidade de fertilizantes vendidos no primeiro trimestre
df_second_quarter = mtm_all_months
df_second_quarter = df_second_quarter[df_second_quarter["Mês"].dt.quarter == 2].reset_index(drop = True)

# Agrupar as quantidades vendidas em cada mês desse primeiro trimestre
df_second_quarter = pd.DataFrame(df_second_quarter.groupby(["Produto", "Mês"])["Quantidade (L)"].sum()).reset_index()

In [ ]:
# Criação do gráfico de barras separados por cada fertilizante e suas respectivas
# vendas em cada mês do primeiro trimestre
second_quarter = px.bar(
              df_second_quarter, 
              x='Mês', 
              y='Quantidade (L)', 
              color='Produto',
              text_auto = '.2s',
              facet_col = 'Produto',
              facet_col_wrap= 6,
              facet_row_spacing = 0.04,
              facet_col_spacing = 0.02,
              #width = 1200,
              height = 2400,
              title='<b>Vendas de fertilizantes mensal do segundo trimestre de 2021</b>'
              )



# Configurações Gerais

# Alteração do tamanho e alinhamento do título do gráfico
second_quarter.update_layout(title=dict(x=0.5, font=dict(size=24)))

# Remoção da string "Product_plot=" do título de cada subplot
second_quarter.for_each_annotation(lambda a: a.update(text=a.text.replace("Produto=", "")))

# Define qual a localização do texto de cada barra
#second_quarter.update_traces(textposition='outside')



# Configurações da legenda

# Esconder a legenda do gráfico
second_quarter.update_traces(showlegend=False)
second_quarter.update(layout_showlegend=False)



# Configurações do eixo x 

# Mudar o formato da data para somente o mês (%b)
second_quarter.update_xaxes(tickformat="%b")

# Mostrar os meses em todos subplots
second_quarter.update_xaxes(showticklabels=True)

# Mostrar sempre cada um dos meses
second_quarter.update_xaxes(dtick="M1")

# Aumentar a distancia dos meses ao eixo x
second_quarter.update_xaxes(linewidth = 3)

# Remoção do título do eixo x em cada subplot
second_quarter.for_each_xaxis(lambda x: x.update(title = ''))



# Configurações do eixo y

# Remoção do título do eixo y em cada subplot
second_quarter.for_each_yaxis(lambda y: y.update(title = ''))

second_quarter.show()

### Média e soma de cada fertilizante vendido no segundo trimestre
Criação de dois novos dataframes em que mostram as quantidades somadas de cada fertilizante vendido no segundo trimestre e também a média de cada um deles.

In [ ]:
# Dataframe que mostra a quantidade vendida total de cada fertilizante no
# segundo trimestre
df_second_quarter_sum = pd.DataFrame(df_second_quarter.groupby("Produto")["Quantidade (L)"].sum()).round()
df_second_quarter_sum = df_second_quarter_sum.sort_values("Quantidade (L)", ascending = False).reset_index()
df_second_quarter_sum.head(10)

In [ ]:
# Criação do gráfico de barras separados por cada fertilizante
second_quarter_sum_graphic = px.bar(
              df_second_quarter_sum, 
              x='Produto', 
              y='Quantidade (L)', 
              text_auto = '.2s',
              title='<b>Fertilizantes vendidos no segundo trimestre de 2021</b>'
              )

# Alteração do tamanho e alinhamento do título do gráfico
second_quarter_sum_graphic.update_layout(title=dict(x=0.5, font=dict(size=24)))

second_quarter_sum_graphic.show()

In [ ]:
# Dataframe que mostra a média mensal vendida de cada fertilizante no
# segundo trimestre
df_second_quarter_avg = pd.DataFrame(df_second_quarter.groupby("Produto")["Quantidade (L)"].mean()).round()
df_second_quarter_avg = df_second_quarter_avg.sort_values("Quantidade (L)", ascending = False).reset_index()
df_second_quarter_avg.head(10)

## Fertilizantes vendidos no terceiro trimestre de 20021

In [ ]:
# Dataframe que mostra a quantidade de fertilizantes vendidos no primeiro trimestre
df_third_quarter = mtm_all_months
df_third_quarter = df_third_quarter[df_third_quarter["Mês"].dt.quarter == 3].reset_index(drop = True)

# Agrupar as quantidades vendidas em cada mês desse primeiro trimestre
df_third_quarter = pd.DataFrame(df_third_quarter.groupby(["Produto", "Mês"])["Quantidade (L)"].sum()).reset_index()

In [ ]:
# Criação do gráfico de barras separados por cada fertilizante e suas respectivas
# vendas em cada mês do primeiro trimestre
third_quarter = px.bar(
              df_third_quarter, 
              x='Mês', 
              y='Quantidade (L)', 
              color='Produto',
              text_auto = '.2s',
              facet_col = 'Produto',
              facet_col_wrap= 6,
              facet_row_spacing = 0.04,
              facet_col_spacing = 0.02,
              #width = 1200,
              height = 2400,
              title='<b>Vendas de fertilizantes mensal do terceiro trimestre de 2021</b>'
              )



# Configurações Gerais

# Alteração do tamanho e alinhamento do título do gráfico
third_quarter.update_layout(title=dict(x=0.5, font=dict(size=24)))

# Remoção da string "Product_plot=" do título de cada subplot
third_quarter.for_each_annotation(lambda a: a.update(text=a.text.replace("Produto=", "")))

# Define qual a localização do texto de cada barra
#second_quarter.update_traces(textposition='outside')



# Configurações da legenda

# Esconder a legenda do gráfico
third_quarter.update_traces(showlegend=False)
third_quarter.update(layout_showlegend=False)



# Configurações do eixo x 

# Mudar o formato da data para somente o mês (%b)
third_quarter.update_xaxes(tickformat="%b")

# Mostrar os meses em todos subplots
third_quarter.update_xaxes(showticklabels=True)

# Mostrar sempre cada um dos meses
third_quarter.update_xaxes(dtick="M1")

# Aumentar a distancia dos meses ao eixo x
third_quarter.update_xaxes(linewidth = 3)

# Remoção do título do eixo x em cada subplot
third_quarter.for_each_xaxis(lambda x: x.update(title = ''))



# Configurações do eixo y

# Remoção do título do eixo y em cada subplot
third_quarter.for_each_yaxis(lambda y: y.update(title = ''))

third_quarter.show()

### Média e soma de cada fertilizante vendido no terceiro trimestre
Criação de dois novos dataframes em que mostram as quantidades somadas de cada fertilizante vendido no terceiro trimestre e também a média de cada um deles.

In [ ]:
# Dataframe que mostra a quantidade vendida total de cada fertilizante no
# terceiro trimestre
df_third_quarter_sum = pd.DataFrame(df_third_quarter.groupby("Produto")["Quantidade (L)"].sum()).round()
df_third_quarter_sum = df_third_quarter_sum.sort_values("Quantidade (L)", ascending = False).reset_index()
df_third_quarter_sum.head(10)

In [ ]:
# Criação do gráfico de barras separados por cada fertilizante
third_quarter_sum_graphic = px.bar(
              df_third_quarter_sum, 
              x='Produto', 
              y='Quantidade (L)', 
              text_auto = '.2s',
              title='<b>Fertilizantes vendidos no terceiro trimestre de 2021</b>'
              )

# Alteração do tamanho e alinhamento do título do gráfico
third_quarter_sum_graphic.update_layout(title=dict(x=0.5, font=dict(size=24)))

third_quarter_sum_graphic.show()

In [ ]:
# Dataframe que mostra a média mensal vendida de cada fertilizante no
# terceiro trimestre
df_third_quarter_avg = pd.DataFrame(df_third_quarter.groupby("Produto")["Quantidade (L)"].mean()).round()
df_third_quarter_avg = df_third_quarter_avg.sort_values("Quantidade (L)", ascending = False).reset_index()
df_third_quarter_avg.head(10)

## Fertilizantes vendidos no quarto trimestre de 20021

In [ ]:
# Dataframe que mostra a quantidade de fertilizantes vendidos no quarto trimestre
df_fourth_quarter = mtm_all_months
df_fourth_quarter = df_fourth_quarter[df_fourth_quarter["Mês"].dt.quarter == 4].reset_index(drop = True)

# Agrupar as quantidades vendidas em cada mês desse primeiro trimestre
df_fourth_quarter = pd.DataFrame(df_fourth_quarter.groupby(["Produto", "Mês"])["Quantidade (L)"].sum()).reset_index()

In [ ]:
# Criação do gráfico de barras separados por cada fertilizante e suas respectivas
# vendas em cada mês do primeiro trimestre
fourth_quarter = px.bar(
              df_fourth_quarter, 
              x='Mês', 
              y='Quantidade (L)', 
              color='Produto',
              text_auto = '.2s',
              facet_col = 'Produto',
              facet_col_wrap= 6,
              facet_row_spacing = 0.04,
              facet_col_spacing = 0.02,
              #width = 1200,
              height = 2400,
              title='<b>Vendas de fertilizantes mensal do quarto trimestre de 2021</b>'
              )



# Configurações Gerais

# Alteração do tamanho e alinhamento do título do gráfico
fourth_quarter.update_layout(title=dict(x=0.5, font=dict(size=24)))

# Remoção da string "Product_plot=" do título de cada subplot
fourth_quarter.for_each_annotation(lambda a: a.update(text=a.text.replace("Produto=", "")))

# Define qual a localização do texto de cada barra
#second_quarter.update_traces(textposition='outside')



# Configurações da legenda

# Esconder a legenda do gráfico
fourth_quarter.update_traces(showlegend=False)
fourth_quarter.update(layout_showlegend=False)



# Configurações do eixo x 

# Mudar o formato da data para somente o mês (%b)
fourth_quarter.update_xaxes(tickformat="%b")

# Mostrar os meses em todos subplots
fourth_quarter.update_xaxes(showticklabels=True)

# Mostrar sempre cada um dos meses
fourth_quarter.update_xaxes(dtick="M1")

# Aumentar a distancia dos meses ao eixo x
fourth_quarter.update_xaxes(linewidth = 3)

# Remoção do título do eixo x em cada subplot
fourth_quarter.for_each_xaxis(lambda x: x.update(title = ''))



# Configurações do eixo y

# Remoção do título do eixo y em cada subplot
fourth_quarter.for_each_yaxis(lambda y: y.update(title = ''))

fourth_quarter.show()

### Média e soma de cada fertilizante vendido no quarto trimestre
Criação de dois novos dataframes em que mostram as quantidades somadas de cada fertilizante vendido no quarto trimestre e também a média de cada um deles.

In [ ]:
# Dataframe que mostra a quantidade vendida total de cada fertilizante no
# quarto trimestre
df_fourth_quarter_sum = pd.DataFrame(df_fourth_quarter.groupby("Produto")["Quantidade (L)"].sum()).round()
df_fourth_quarter_sum = df_fourth_quarter_sum.sort_values("Quantidade (L)", ascending = False).reset_index()
df_fourth_quarter_sum.head(10)

In [ ]:
# Criação do gráfico de barras separados por cada fertilizante
fourth_quarter_sum_graphic = px.bar(
              df_fourth_quarter_sum, 
              x='Produto', 
              y='Quantidade (L)', 
              text_auto = '.2s',
              title='<b>Fertilizantes vendidos no quarto trimestre de 2021</b>'
              )

# Alteração do tamanho e alinhamento do título do gráfico
fourth_quarter_sum_graphic.update_layout(title=dict(x=0.5, font=dict(size=24)))

fourth_quarter_sum_graphic.show()

In [ ]:
# Dataframe que mostra a média mensal vendida de cada fertilizante no
# quarto trimestre
df_fourth_quarter_avg = pd.DataFrame(df_fourth_quarter.groupby("Produto")["Quantidade (L)"].mean()).round(0)
df_fourth_quarter_avg = df_fourth_quarter_avg.sort_values("Quantidade (L)", ascending = False).reset_index()
df_fourth_quarter_avg.head(10)